In [2]:
# System libraries:
%load_ext autoreload
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import pandas as pd
import json
import nltk
import numpy as np
import os
import sys
from IPython.core.interactiveshell import InteractiveShell
from pymystem3 import Mystem
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from datetime import datetime
import re
import gensim.downloader as api
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
InteractiveShell.ast_node_interactivity = "all"

In [8]:
data = pd.read_csv('data/processed/yle_fi_lemmas.csv', converters = {'lemmas_content' : eval, 'lemmas_headline':eval})
data.iloc[:5]

,Unnamed: 0,headline,content,datePublished,lemmas_content,year
0,0,Kaukasian konflikti: Aseissa Georgia ja Venäjä...,Kaukasian konflikti: Aseissa Georgia ja Venäjä...,2008-08-11T11:03:41+0300,"[kaukasia, konflikti, ase, georg, venäjä, tuke...",2008
1,1,Otteita Venäjän-tuntijoiden vastauksista,Otteita Venäjän-tuntijoiden vastauksista\n\nTä...,2007-01-14T12:46:58+0200,"[ote, venäjä, vasta, tiivistelmä, koota, ote, ...",2007
2,2,Etelä-Ossetian ruutitynnyri räjähti viimein,Etelä-Ossetian ruutitynnyri räjähti viimein\n\...,2008-08-12T21:33:32+0300,"[etelä-ossetia, ruuti, räjähtää, viimein, perj...",2008
3,3,Yle Uutiset seuraa Ukrainan kriisiä hetki hetk...,Yle Uutiset seuraa Ukrainan kriisiä hetki hetk...,2014-03-18T09:06:59+0200,"[yle, uutinen, seura, ukraina, kriisi, hetki, ...",2014
4,4,Saksan ulkopolitiikan täyskäännös,Saksan ulkopolitiikan täyskäännös\n\nTuskin mi...,2014-10-02T12:30:30+0300,"[saksa, täyskäännös, tuska, mikään, muu, euroo...",2014


In [9]:
sentences = data.lemmas_content
model = gensim.models.Word2Vec(sentences, min_count=1)


2018-05-30 10:13:03,455 : INFO : collecting all words and their counts
2018-05-30 10:13:03,460 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-05-30 10:13:04,002 : INFO : PROGRESS: at sentence #10000, processed 2297196 words, keeping 41429 word types
2018-05-30 10:13:04,396 : INFO : PROGRESS: at sentence #20000, processed 4006068 words, keeping 66820 word types
2018-05-30 10:13:04,769 : INFO : PROGRESS: at sentence #30000, processed 5683074 words, keeping 90594 word types
2018-05-30 10:13:05,075 : INFO : PROGRESS: at sentence #40000, processed 7420611 words, keeping 115179 word types
2018-05-30 10:13:05,403 : INFO : PROGRESS: at sentence #50000, processed 9166757 words, keeping 145130 word types
2018-05-30 10:13:05,751 : INFO : PROGRESS: at sentence #60000, processed 11001060 words, keeping 168064 word types
2018-05-30 10:13:06,060 : INFO : collected 185506 word types from a corpus of 12599269 raw words and 68472 sentences
2018-05-30 10:13:06,061 : INFO 

2018-05-30 10:14:05,288 : INFO : EPOCH 3 - PROGRESS: at 89.93% examples, 661862 words/s, in_qsize 6, out_qsize 0
2018-05-30 10:14:06,289 : INFO : EPOCH 3 - PROGRESS: at 97.20% examples, 674427 words/s, in_qsize 5, out_qsize 0
2018-05-30 10:14:06,736 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-30 10:14:06,746 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-05-30 10:14:06,755 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-30 10:14:06,756 : INFO : EPOCH - 3 : training on 12599269 raw words (12057659 effective words) took 17.8s, 678974 effective words/s
2018-05-30 10:14:07,768 : INFO : EPOCH 4 - PROGRESS: at 2.32% examples, 616218 words/s, in_qsize 5, out_qsize 0
2018-05-30 10:14:08,785 : INFO : EPOCH 4 - PROGRESS: at 7.09% examples, 661586 words/s, in_qsize 5, out_qsize 0
2018-05-30 10:14:09,805 : INFO : EPOCH 4 - PROGRESS: at 12.78% examples, 646766 words/s, in_qsize 5, out_qsize 0
2018-05-30 10:14:10,829 

In [32]:
model.wv.most_similar(positive=['naapuri'],topn=10)

[('länsi', 0.6412229537963867),
 ('itä', 0.5725439786911011),
 ('suurvalta', 0.5677061080932617),
 ('vīcīna', 0.5491490364074707),
 ('venäjä', 0.5239830613136292),
 ('vihollinen', 0.515234649181366),
 ('toisaalta', 0.5106375813484192),
 ('lähialue', 0.49187231063842773),
 ('viha', 0.4911051392555237),
 ('suhde', 0.4877402186393738)]

In [29]:
words = ['venäjä','usa']
number_of_results = 25

print("Nearest words:")
model.wv.most_similar(positive=words ,topn=number_of_results)

Nearest words:


[('yhdysvallat', 0.7446652054786682),
 ('kiina', 0.6143582463264465),
 ('kanada', 0.6052343249320984),
 ('japani', 0.5615869760513306),
 ('maa', 0.5432892441749573),
 ('valko-venäjä', 0.5337971448898315),
 ('suomia', 0.5253757238388062),
 ('ruotsi', 0.505166232585907),
 ('kuuba', 0.5051059126853943),
 ('isäntä', 0.48110100626945496),
 ('suomi', 0.4791686534881592),
 ('brasilia', 0.47670450806617737),
 ('iso-britannia', 0.47642916440963745),
 ('kazakstan', 0.46718963980674744),
 ('etelä-korea', 0.4633820950984955),
 ('australia', 0.45188766717910767),
 ('nigeria', 0.45019039511680603),
 ('tanska', 0.4425284266471863),
 ('saksa', 0.4422125518321991),
 ('jenkki', 0.43623030185699463),
 ('nhl-ryhmän', 0.43525123596191406),
 ('slovakia', 0.4347362220287323),
 ('etelä-afrikka', 0.4339946508407593),
 ('ranska', 0.43015986680984497),
 ('hallita', 0.42894017696380615)]

In [18]:
words_to_compare = ('pietari', 'turku')

print("Word similarity:")
model.wv.similarity(*words_to_compare)

Word similarity:


0.42047345716505585

In [27]:
positive_terms = ['suomi', 'demokraatti']
negative_terms = ['venäjä']

print("Vector math:")
model.wv.most_similar(positive=positive_terms, negative=negative_terms, topn=5)


Vector math:


[('republikaani', 0.6388195753097534),
 ('esivaali', 0.5393120050430298),
 ('republikaaninen', 0.5293186902999878),
 ('parlamentarismi', 0.5172135829925537),
 ('ihanne', 0.511397659778595)]